# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [57]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [40]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

In [4]:
# load data from database
def load_data():
    from sqlalchemy import create_engine
    engine = create_engine('sqlite:///DisasterResponse.db')
    df = pd.read_sql_table('disaster', engine)  
    X = df.message.values
    y = df.iloc[:,4:]
    category_names = df.columns
    return X, y, category_names

X, y, category_names = load_data()

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):    
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    
    tokens = [lemmatizer.lemmatize(word, pos='v') for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

In [11]:
from sklearn.multioutput import MultiOutputClassifier

model = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

X, y, category_names = load_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [28]:
for i in range(36):
    from sklearn.metrics import classification_report
    y_predict = y_pred[:,i].reshape(-1,1)
    y_test1 = y_test.iloc[:,i]
    print(category_names[i+4])
    print(classification_report(y_test1, y_predict, target_names=['not 1', '1']))

related
             precision    recall  f1-score   support

      not 1       0.63      0.48      0.54      1500
          1       0.85      0.91      0.88      5007

avg / total       0.80      0.81      0.80      6545

request
             precision    recall  f1-score   support

      not 1       0.90      0.98      0.94      5446
          1       0.80      0.43      0.56      1099

avg / total       0.88      0.89      0.87      6545

offer
             precision    recall  f1-score   support

      not 1       0.99      1.00      1.00      6506
          1       0.00      0.00      0.00        39

avg / total       0.99      0.99      0.99      6545

aid_related
             precision    recall  f1-score   support

      not 1       0.75      0.85      0.80      3816
          1       0.74      0.60      0.67      2729

avg / total       0.75      0.75      0.74      6545

medical_help
             precision    recall  f1-score   support

      not 1       0.93      0.99      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 2
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 2
  .format(len(labels), len(target_names))


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
parameters = {
  
        'clf__estimator__n_estimators': [20, 40],
        'clf__estimator__min_samples_split': [2, 4],
        'clf__estimator__max_depth': [4,6]
    }

cv = GridSearchCV(model, param_grid=parameters)

cv.fit(X_train, y_train)
y_pred1 = model.predict(X_test)
print(cv.best_params_)

In [26]:

best_param = {'clf__estimator__max_depth': 4, 'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 40}
model1 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(max_depth=4, min_samples_split=2, n_estimators=40)))
])
model1.fit(X_train, y_train)
y_pred1 = model1.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [31]:
for i in range(36):
    from sklearn.metrics import classification_report
    y_predict1 = y_pred1[:,i].reshape(-1,1)
    y_test1 = y_test.iloc[:,i]
    print(category_names[i+4])
    print(classification_report(y_test1, y_predict1, target_names=['not 1', '1']))

related
             precision    recall  f1-score   support

      not 1       0.00      0.00      0.00      1500
          1       0.77      1.00      0.87      5007

avg / total       0.59      0.77      0.66      6545

request
             precision    recall  f1-score   support

      not 1       0.83      1.00      0.91      5446
          1       0.00      0.00      0.00      1099

avg / total       0.69      0.83      0.76      6545

offer
             precision    recall  f1-score   support

      not 1       0.99      1.00      1.00      6506
          1       0.00      0.00      0.00        39

avg / total       0.99      0.99      0.99      6545

aid_related
             precision    recall  f1-score   support

      not 1       0.58      1.00      0.74      3816
          1       0.94      0.01      0.01      2729

avg / total       0.73      0.59      0.44      6545

medical_help
             precision    recall  f1-score   support

      not 1       0.92      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 2
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 2
  .format(len(labels), len(target_names))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [58]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
model1 = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
   ])),

    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

parameters = {
    'clf__estimator__n_estimators': [20, 40, 60],
    'clf__estimator__learning_rate': [0.05, 0.10]
}

cv1 = GridSearchCV(model1, param_grid=parameters)
model1.fit(X_train, y_train)
print(cv1.best_params_)
y_pred2 = cv1.predict(X_test)


for i in range(36):
    from sklearn.metrics import classification_report
    y_predict2 = y_pred2[:,i].reshape(-1,1)
    y_test1 = y_test.iloc[:,i]
    print(category_names[i+4])
    print(classification_report(y_test1, y_predict2, target_names=['not 1', '1']))

IndexError: list index out of range

### 9. Export your model as a pickle file

In [51]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
import sys
import nltk
nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

def load_data(database_filepath):
    #input: sqlite database filepath containing clean dataset
    #Output: X:messages y: array of 36 individual categories columns, categories_names: Names for the 36 categories
    
    from sqlalchemy import create_engine
    engine = create_engine('sqlite:///' + database_filepath)
    #read in the clean dataset from sqlite database
    df = pd.read_sql_table('disaster_table', engine)
    #extract the messages
    X = df.message.values
    #extract the array of 36 individual categories columns
    y = df.iloc[:,4:]
    #extract the names for the 36 categories
    category_names = df.columns
    return X, y, category_names


def tokenize(text):
    #input: messages text
    #Output: tokenized and lemmatized text
    
    #store stop words
    stop_words = stopwords.words("english")
    #Lemmatizer function
    lemmatizer = WordNetLemmatizer()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    #tokenize the text
    tokens = word_tokenize(text)
    #lemmatize the word tokens if they are not stop words
    tokens = [lemmatizer.lemmatize(word, pos='v') for word in tokens if word not in stop_words]
    return tokens


def build_model():
    from sklearn.multioutput import MultiOutputClassifier
    #Build a machine learning pipeline
    
    pipeline = Pipeline([
        #Convert a collection of text documents to a matrix of lemmatized token counts
        ('vect', CountVectorizer(tokenizer=tokenize)),
        #Transform a count matrix to a normalized tf or tf-idf representation
        ('tfidf', TfidfTransformer()),
        #train a Random Forest Classifier
        ('clf', MultiOutputClassifier(RandomForestClassifier(max_depth=4, min_samples_split=2, n_estimators=40)))
    ])
    
    return pipeline

def evaluate_model(model, X_test, y_test, category_names):
    #input: trained model, test dataset, test labels, names of the 36 categories
    #Output: Print the f1 score, precision and recall for each output category of the dataset
    
    from sklearn.metrics import classification_report
    length_categories = len(category_names)
    #use the trained model to predict on test dataset
    y_pred = model.predict(X_test)
    
    #iterating through the 36 categories columns
    for i in range(length_categories-4):
        #extract the predictions for each column
        y_predict1 = y_pred[:,i].reshape(-1,1)
        #extract the correct labels for each column
        y_test1 = y_test.iloc[:,i]
        print(category_names[i+4])
        #Report the f1 score, precision and recall for each output category of the dataset
        print(classification_report(y_test1, y_predict1, target_names=['not 1', '1']))


def save_model(model, model_filepath):
    #input: trained model, model filepath to save the model to
    #Output: model saved to the specified filepath
    
    import pickle
    pickle.dump(model, open(model_filepath, 'wb'))


def main():
    if len(sys.argv) == 3:
        
        database_filepath, model_filepath = sys.argv[1:]
        print('Loading data...\n    DATABASE: {}'.format(database_filepath))
        #load the clean dataset from SQLite database and extract the messages, array of 36 individual categories columns and name for the 36 categories
        X, Y, category_names = load_data(database_filepath)
        #split the dataset into training and test dataset with test_size set at 0.2
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
        #build the ML model
        print('Building model...')
        model = build_model()
        #fit the model on training dataset
        print('Training model...')
        model.fit(X_train, Y_train)
        #evaluate the model by reporting the f1 score, precision and recall for each output category of the dataset
        print('Evaluating model...')
        evaluate_model(model, X_test, Y_test, category_names)
        #save the trained model to the specified filepath
        print('Saving model...\n    MODEL: {}'.format(model_filepath))
        save_model(model, model_filepath)

        print('Trained model saved!')

    else:
        print('Please provide the filepath of the disaster messages database '\
              'as the first argument and the filepath of the pickle file to '\
              'save the model to as the second argument. \n\nExample: python '\
              'train_classifier.py ../data/DisasterResponse.db classifier.pkl')


if __name__ == '__main__':
    main()

In [ ]:
def model_pipeline():
    from sklearn.multioutput import MultiOutputClassifier
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(CatBoostClassifier()))
    ])

    parameters = {
        'n_estimators': [50, 100, 200],
        'min_data_in_leaf': [2, 3, 4],
        'learning_rate': [0.03,0.06,0.10]
        'max_depth': [6,8,10]
    }

    cv = GridSearchCV(pipeline, param_grid=parameters)

    return cv